In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# 카메라 파라미터 (미리 측정 또는 캘리브레이션)
fx, fy = 724.79, 720.74  # 초점 거리
cx, cy = 322.13, 235.08  # 중심 좌표 (frame size 640x480 기준)

# 실제 객체 크기 (미리 측정한 값, 단위: m)
real_width = {
    'I_P': 0.06,   # 6cm
    }

# 원하는 label 매핑
label_map = {
    'I_P': 'I_P',
    }

# 2차 다항 회귀 함수를 이용한 거리 보정
bbox_w_list = [234, 152, 125, 99, 91, 80, 59, 61]
true_distance_list = [20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0]
coeffs = np.polyfit(bbox_w_list, true_distance_list, deg=2)
def corrected_distance(bbox_w):
    return np.polyval(coeffs, bbox_w)


# YOLOv8 모델 불러오기
model = YOLO('/home/choigh/WS/IP_OB/Installation_point/yolo_data/runs/detect/train3/weights/best.pt')

# 웹캠 열기
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    
    for box in results.boxes:
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        conf = float(box.conf[0])

        # 객체가 우리가 추적하고 싶은 것인지 확인
        if label in real_width:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            bbox_cx = (x1 + x2) / 2
            bbox_cy = (y1 + y2) / 2
            bbox_w = x2 - x1

            # 거리(Z) 추정
            Z = corrected_distance(bbox_w)
            X = (bbox_cx - cx) * Z / fx
            Y = (bbox_cy - cy) * Z / fy

            name = label_map[label]
            print(f"[{name}] → X: {X:.2f}m, Y: {Y:.2f}m, Z: {Z:.2f}m")

            # 시각화
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, f"{name} ({X:.2f},{Y:.2f},{Z:.2f})m", (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

    cv2.imshow("YOLOv8 + 3D", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 1 I_P, 51.2ms
Speed: 5.7ms preprocess, 51.2ms inference, 169.8ms postprocess per image at shape (1, 3, 480, 640)
[I_P] → X: -7.44m, Y: 3.03m, Z: 52.01m

0: 480x640 1 I_P, 7.8ms
Speed: 2.3ms preprocess, 7.8ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
[I_P] → X: -7.07m, Y: 2.95m, Z: 50.15m

0: 480x640 1 I_P, 11.3ms
Speed: 1.6ms preprocess, 11.3ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
[I_P] → X: -7.19m, Y: 2.95m, Z: 50.76m

0: 480x640 1 I_P, 11.0ms
Speed: 1.9ms preprocess, 11.0ms inference, 4.3ms postprocess per image at shape (1, 3, 480, 640)
[I_P] → X: -7.07m, Y: 2.92m, Z: 50.15m

0: 480x640 1 I_P, 7.6ms
Speed: 1.4ms preprocess, 7.6ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)
[I_P] → X: -7.19m, Y: 2.99m, Z: 50.76m

0: 480x640 1 I_P, 9.2ms
Speed: 1.3ms preprocess, 9.2ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)
[I_P] → X: -7.31m, Y: 3.06m, Z: 51.38m

0: 480x640 1 I_P, 10.0ms
Speed:

In [ ]:
# 2차 다항 회귀 함수를 이용한 거리 보정 데이터 수집 코드

import cv2
from ultralytics import YOLO

model = YOLO('/home/cgh/workspace/project1/Installation_point/yolo_data/runs/detect/train3/weights/best.pt')  # 학습한 best.pt

cap = cv2.VideoCapture(0)

# 수집된 데이터 저장용 리스트
bbox_width_list = []
real_distance_list = []

print("[INFO] 's' 키를 눌러 bbox_w 측정, 실측 거리 입력")
print("[INFO] 'q' 키를 누르면 종료됩니다")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    for box in results.boxes:
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        bbox_w = x2 - x1

        # 시각화
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} w={bbox_w}px", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        break  # 첫 번째 물체만 사용

    cv2.imshow('Measurement Mode', frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('s'):  # save
        print(f"\n[BBOX WIDTH] → {bbox_w}px")
        cm = input("[INPUT] 실제 거리(cm)를 입력하세요: ")
        try:
            cm = float(cm)
            bbox_width_list.append(bbox_w)
            real_distance_list.append(cm)
            print(f"[SAVED] bbox_w: {bbox_w}, real_distance: {cm}")
        except:
            print("[ERROR] 숫자로 입력하세요!")

    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 결과 출력
print("\n[RESULT]")
print("bbox_width_list =", bbox_width_list)
print("real_distance_list =", real_distance_list)


[INFO] 's' 키를 눌러 bbox_w 측정, 실측 거리 입력
[INFO] 'q' 키를 누르면 종료됩니다

0: 480x640 1 I_P, 8.3ms
Speed: 3.2ms preprocess, 8.3ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 I_P, 8.9ms
Speed: 1.1ms preprocess, 8.9ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 I_P, 8.2ms
Speed: 3.9ms preprocess, 8.2ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 I_P, 9.3ms
Speed: 1.6ms preprocess, 9.3ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 I_P, 7.4ms
Speed: 3.3ms preprocess, 7.4ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 I_P, 9.0ms
Speed: 1.2ms preprocess, 9.0ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 I_P, 7.3ms
Speed: 1.5ms preprocess, 7.3ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 I_P, 10.2ms
Speed: 1.7ms preprocess, 10.2ms inference, 1.9ms postprocess per i